In [1]:
from scrape_data import get_runner_ids, get_splits, load_nyc_data
import pandas as pd
# import requests
# import json

# x = get_runner_ids(eventCode="M2023", place1=51340, place2=51348)
# get_splits(runner_id=43928778)  # AssertionError

# tables = load_nyc_data(last_runner=53520, eventCode="M2019")
# tables = load_data(last_runner=24949, eventCode="M2021") # 216mins
# tables = load_data(last_runner=47745, eventCode="M2022")
# tables = load_data(last_runner=51348, eventCode="M2023")
# tables = load_data(last_runner=55270, eventCode="M2024")
# pd.concat(tables).to_csv("raw_data/nyc/nyc_names21.csv", index=False)

In [ ]:
# pd.concat(tables).to_csv("raw_data/nyc/nyc_names19.csv", index=False)

In [2]:
import time

def loop_attempts(a_dict, n, year=23):
    for i in range(n):
        a_dict = attempt_scrape_iteration(a_dict, year=year)
        num_left = len([k for k, v in a_dict.items() if v is None])
        print(f"{i}th cooldown, 30 seconds..........num left = {num_left}")
        pd.DataFrame([v for v in a_dict.values() if v is not None]).to_csv(f"raw_data/nyc/nyc_times{year}.csv")
        if num_left == 0:
            print("done!!!!")
            return a_dict
        time.sleep(30)

    return a_dict

def attempt_scrape_iteration(adict, year=23):
    ids = [k for k, v in adict.items() if v is None]
    for idx, id in enumerate(ids):
        try:
            splits = get_splits(id)['details']['splitResults']
            dt = {dct["splitCode"]: dct["time"] for dct in splits}
            ser = pd.Series(dt)
            ser["id"] = id
            adict[id] = ser

            if idx % 100 == 0:
                print("loaded", idx)
                pd.DataFrame([v for v in adict.values() if v is not None]).to_csv(f"raw_data/nyc/nyc_times{year}.csv")

            if idx % 100 == 0:
                time.sleep(6.6)
            if idx % 44 == 0:
                time.sleep(4.4)
            if idx % 55 == 0:
                time.sleep(5.5)

            # if idx == 10:
            #     break

        except Exception as e:
            print("idx=", idx, ":", e)
            break
    
    return adict

In [56]:
#del
import json
import requests

# def get_splits(runner_id=42948497):
#     url = 'https://rmsprodapi.nyrr.org/api/v2/runners/resultDetails/'
#     header = {"content-type": "application/json;charset=UTF-8"}
#     response = requests.get(url, headers=header, json={"runnerId": str(runner_id)}, allow_redirects=True)
#     assert response.status_code == 200, f"Response Code = {response.status_code}"
#     d = json.loads(response.text)
#     assert d["details"], f"No data specified, d = {d}"
#     return d


year = 22
data_load21 = set(pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv")["runnerId"])
left = data_load21 - set(pd.read_csv("raw_data/nyc/nyc_times22_new.csv")["id"])
init_dict21 = {id: None for id in data_load21}
ids = data_load21
# url = 'https://rmsprodapi.nyrr.org/api/v2/runners/resultDetails/'
# header = {"content-type": "application/json;charset=UTF-8"}
# s = requests.Session()
# for id in ids:
#     response = s.post(url, headers=header, json={"runnerId":id}, allow_redirects=True, timeout=10)
len(ids), len(left)

(47745, 37144)

In [79]:
combined = pd.concat([pd.read_csv("raw_data/nyc/nyc_times22.csv"), pd.read_csv("raw_data/nyc/nyc_times22_.csv")])
combined.drop("Unnamed: 0", axis=1).drop_duplicates().to_csv("raw_data/nyc/nyc_times22.csv", index=False)
# combined.drop("Unnamed: 0", axis=1).drop_duplicates()

In [81]:

# pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv").merge(combined.drop("Unnamed: 0", axis=1).drop_duplicates(), left_on="runnerId", right_on="id")


d_load = pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv")
times = pd.read_csv(f"raw_data/nyc/nyc_times{year}.csv")
d_load.merge(times, left_on="runnerId", right_on="id").to_csv(f"raw_data/nyc/nyc{year}.csv")

In [ ]:
year = 21
data_load21 = pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv")
init_dict21 = {id: None for id in list(data_load21["runnerId"])}
adding_dict21 = loop_attempts(init_dict21, n=100, year=year)

{39876379: None, 39846321: None}

In [ ]:
times21 = pd.DataFrame([v for v in adding_dict21.values() if v is not None])
data_load21.merge(times21, left_on="runnerId", right_on="id")#.to_csv("raw_data/nyc/nyc21.csv")

In [ ]:
# ids = pd.read_csv("raw_data/nyc/nyc_names22.csv")["runnerId"].values
# ids_done = pd.read_csv("raw_data/nyc/nyc_times22.csv")["id"].values
# ids_left = [i for i in ids if i not in ids_done]

In [ ]:
year = 22
data_load22 = pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv")
init_dict22 = {id: None for id in list(data_load22["runnerId"])}
# adding_dict22 = loop_attempts(init_dict22, n=100, year=year)

In [ ]:
data_load22["runnerId"][16]

In [ ]:
s = pd.Series({dct["splitCode"]: dct["time"] for dct in get_splits(id)['details']['splitResults']})
s['id']

In [ ]:
marks = ["5K", "10K", "15K", "20K", "HALF", "25K", "30K", "35K", "40K", "MAR"]

In [ ]:
test_id =d2['items'][0]["runnerId"]
d3 = get_splits(runner_id = test_id)
d3

In [ ]:
d3["details"]

In [ ]:
import json
# url = 'https://rmsprodapi.nyrr.org/api/v2/runners/eventRunner'
url = 'https://rmsprodapi.nyrr.org/api/v2/runners/resultDetails'
j = {"eventCode": "M2024", "bib": "4"}
header = {"content-type": "application/json;charset=UTF-8"}
response = requests.post(url, headers=header, json=j, allow_redirects=True)
print(response)

In [ ]:
json.loads(response.text)["details"]

Chicago

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def scrape_chi_data(yr=23):
    event_param = {23: "MAR_9TGG963812D", 22: "MAR_9TGG9638119", 21: "MAR_9TGG9638F1", 19: "MAR_999999107FA31100000000C9"}
    links_list = pd.read_csv(f"raw_data/chicago/ids{year}.csv")['0'].values

    results = []
    for idx, person in enumerate(links_list[:100]):
        params = {"content" : "detail", "idp" : person, "lang" : "EN", "event" : event_param[year],}
        response = requests.post("https://chicago-history.r.mikatiming.com/2023", params=params)
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find("table", attrs={"class":"table table-condensed table-striped"})
        rows = table.findAll("tr")
        splits = [[t.text.strip() for t in row.findAll(["th", "td"])][2] for row in rows[1:]]
        rows2 = soup.findAll(name="table", attrs={"class":"table table-condensed"})[1]#.findAll("th")
        info = [row.text.strip() for row in rows2.findAll(["th", "td"])][1::2]
        results.append(info + splits)
        print(idx)

    mks = [[t.text.strip() for t in row.findAll(["th", "td"])][0] for row in rows[1:]]
    cols = [row.text.strip() for row in rows2.findAll(["th", "td"])][::2]
    pd.DataFrame(results, columns = cols + mks).to_csv(f"raw_data/chicago/chi{yr}.csv")


In [ ]:
pd.DataFrame(results, columns = cols + mks).to_csv("")

In [ ]:
event_ = {23: "MAR_9TGG963812D", 22: "MAR_9TGG9638119", 21: "MAR_9TGG9638F1", 19: "MAR_999999107FA31100000000C9"}

# params = {
#     "content" : "detail",
#     # "fpid" : "search",
#     # "pid" : "search",
#     "idp" : "9TGG9638284476",
#     "lang" : "EN",
#     "event" : "MAR_9TGG963812D",
#     # "lang" : "EN_CAP",
#     # "search_event" : "ALL_EVENT_GROUP_2022",
# }

params = {"content" : "detail", "idp" : "9TGG9638284476", "lang" : "EN", "event" : "MAR_9TGG963812D",}
response = requests.post("https://chicago-history.r.mikatiming.com/2023", params=params)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find("table", attrs={"class":"table table-condensed table-striped"})
rows = table.findAll("tr")
splits = [[t.text.strip() for t in row.findAll(["th", "td"])][2] for row in rows[1:]]
splits

In [ ]:
pd.DataFrame(splits_list, columns=[[t.text.strip() for t in row.findAll(["th", "td"])][0] for row in rows[1:]])
